<a href="https://colab.research.google.com/github/sheelapravalika/AI_CS_mokshasaireddy005/blob/main/SLM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sheelapravalika/AI_CS_mokshasaireddy005/blob/main/SLM1.ipynb)


In [ ]:
!pip install -U datasets setfit scikit-learn pandas matplotlib seaborn


In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from setfit import SetFitModel, SetFitTrainer

In [ ]:
# Load dataset
dataset = load_dataset("sonnh-tech1/cic-ids-2017", "binary")
df = dataset["train"].to_pandas()

# Reduce for faster training
df = df[['FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'Label']]
df = df.sample(n=20000, random_state=42) # Increased sample size

# Encode Label to numeric (BENIGN=0, ATTACK=1)
df['Label'] = df['Label'].astype('category').cat.codes

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/92.8M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2522362 [00:00<?, ? examples/s]

In [ ]:
# The 'text' column is now created after filtering in cell ZwD78B4dLnTC

In [ ]:
# Get counts of each label
label_counts = df['Label'].value_counts()

# Filter out labels with less than 2 samples
valid_labels = label_counts[label_counts >= 2].index

# Filter the dataframe to keep only rows with valid labels
filtered_df = df[df['Label'].isin(valid_labels)].copy() # Added .copy() to avoid SettingWithCopyWarning

# Create 'text' column
filtered_df['text'] = filtered_df[['FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets']].astype(str).agg(' '.join, axis=1)
filtered_df = filtered_df[['text', 'Label']]

# Split
train_df, test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['Label'], random_state=42)

# Rename 'Label' column to 'label'
train_df = train_df.rename(columns={'Label': 'label'})
test_df = test_df.rename(columns={'Label': 'label'})

# Fix numpy.int64 issue and ensure 'label' is integer type
train_df["label"] = train_df["label"].astype(int)
test_df["label"] = test_df["label"].astype(int)

# Convert to Hugging Face Dataset with explicit string type for 'text'
from datasets import Dataset, Value, Features
features = Features({'text': Value('string'), 'label': Value('int64'), '__index_level_0__': Value('int64')})
train_dataset = Dataset.from_pandas(train_df, features=features).remove_columns(['__index_level_0__'])
test_dataset = Dataset.from_pandas(test_df, features=features).remove_columns(['__index_level_0__'])

In [ ]:
# Load a pre-trained Sentence Transformer model
from sentence_transformers import SentenceTransformer
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L3-v2")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
pip install -U setfit

In [ ]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    batch_size=16,
    num_iterations=10
)

/tmp/ipython-input-13-193993251.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [ ]:
# ✅ Train — this now works without error!
trainer.train()

***** Running training *****
  Num unique pairs = 320000
  Batch size = 16
  Num epochs = 1
/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sheelapravalika13 (sheelapravalika13-defence-research-and-development-organ) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


Step,Training Loss
1,0.252000
50,0.251300
100,0.258200
150,0.243900
200,0.236600
250,0.230400
300,0.216900
350,0.205400
400,0.190100
450,0.183200


TypeError: Wrong key type: '12322' of type '<class 'numpy.int64'>'. Expected one of int, slice, range, str or Iterable.

In [ ]:
import nbformat

# Step 1: Upload your broken notebook again
from google.colab import files
uploaded = files.upload()

# Replace this with the actual uploaded filename
filename = list(uploaded.keys())[0]

# Step 2: Load and clean
with open(filename, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# ✅ Step 3: Clean ONLY metadata.widgets (KEEP outputs and execution_count)
for cell in nb.cells:
    if 'metadata' in cell:
        cell['metadata'] = {k: v for k, v in cell['metadata'].items() if k != 'widgets'}

# ✅ Step 4: Clear notebook-level metadata.widgets (if present)
if 'widgets' in nb.get('metadata', {}):
    del nb['metadata']['widgets']

# Step 5: Save cleaned notebook
cleaned_filename = filename.replace('.ipynb', '_fixed.ipynb')
with open(cleaned_filename, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print(f"✅ Cleaned notebook saved (outputs preserved) as {cleaned_filename}")
files.download(cleaned_filename)